- 함수형 API (Functional API) -

Sequential API - 입력부터 출력까지 일직선으로 연결, 직관적으로 편리함.

Functional API - 여러개의 층 공유, 다양한 종류의 입출력 등 일직선 구조 한계를 보완해줌.

example)

In [3]:
#Sequential API 를 Functional API 로 구현

import tensorflow as tf

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

input_ = Input(shape = (4,))

x = Dense(8, activation = 'relu')(input_)
x = Dense(16, activation = 'relu')(x)
x = Dense(32, activation = 'relu')(x)

output_ = Dense(10, activation = 'softmax')(x)

md = Model(inputs = input_, outputs = output_)
md.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 8)                 40        
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 32)                544       
                                                                 
 dense_3 (Dense)             (None, 10)                330       
                                                                 
Total params: 1,058
Trainable params: 1,058
Non-trainable params: 0
_________________________________________________________________


In [ ]:
example 2)

In [5]:
# 다중 입력 in Functional API

from tensorflow.keras.layers import concatenate

#입력층 1에 대한 신경망
input1 = Input(shape = (4,))

hidden1 = Dense(8, activation = 'relu')(input1)
hidden2 = Dense(16, activation = 'relu')(hidden1)

output1 = Model(inputs = input1, outputs = hidden2)

#입력층 2에 대한 신경망
input2 = Input(shape = (8,))

hidden3 = Dense(8, activation = 'relu')(input2)

output2 = Model(inputs = input2, outputs = hidden3)

#층 연결
result = concatenate([output1.output, output2.output])

#출력층 정의
output = Dense(10, activation = 'softmax')(result)

#최종 모델 구축
md = Model(inputs = [output1.input, output2.input], outputs = output)
md.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense_7 (Dense)                (None, 8)            40          ['input_4[0][0]']                
                                                                                                  
 input_5 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense_8 (Dense)                (None, 16)           144         ['dense_7[0][0]']                
                                                                                            

MNIST CNN modeling exmple in Functional API)

In [14]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D
from tensorflow.keras.layers import Dense, Flatten, Dropout

(xtr, ytr), (xt, yt) = mnist.load_data()

xtr = xtr.reshape(-1, 28,28, 1)
xt = xt.reshape(-1, 28, 28, 1)

xtr = xtr / 255.0
xt = xt / 255.0

In [18]:
input_ = Input(shape = (28, 28, 1))

x = Conv2D(32, 3, activation = 'relu')(input_)
x = Conv2D(64, 3, activation = 'relu')(x)
x = MaxPool2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)

x = Flatten()(x)

x = Dense(128, activation = 'relu')(x)
x = Dropout(0.5)(x)

output_ = Dense(10, activation = 'softmax')(x)
cnn = Model(inputs = input_, outputs = output_)

In [ ]:
#CNN 모델 컴파일 및 학습

cnn.compile(loss = 'sparse_categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(),\
           metrics = ['accuracy'])

hist = cnn.fit(xtr, ytr, batch_size = 128, epochs = 30, validation_data = (xt, yt))

-----------------------------------------------------------------------------------------------

- 전이 학습(Transfer Learning) -

부족한 학습 데이터 문제를 해결하기 위해 등장,

ImageNet 데이터를 활용해 학습된 모델의 가중치를 가져와서, 해결하려는 문제에 맞게 보정해서 사용하는 개념,

사전 학습 모델(pre-trained model) : 큰 데이터셋을 사용해 훈련된 모델.

사전 학습 모델의 구조 : 사전 학습된 feature extractor / 사전 학습된 classifier

파인 튜닝(fine - tuning)

사전 학습 모델의 가중치를 미세하게 조정하는 기법,

많은 연산량으로 인해 CPU 보다는 GPU 를 사용.

transfer learning 활용법)

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

from tensorflow.keras.applications import VGG16, ResNet50, MobileNet, InceptionV3
#tensorflow 에서 제공하는 다양한 사전학습 모델 import

base_md = VGG16(weights = 'imagenet', include_top = False, input_shape = (240, 240, 3))
#input_shape = (widths, heights, channel)
#include_top = False : 특징 추출기만 가져옴, True : 특징 추출기와 분류기 모두 가져옴.
base_md.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 240, 240, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 240, 240, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 240, 240, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 120, 120, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 120, 120, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 120, 120, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 60, 60, 128)       0     

include_top = False 지정하여, conv2D 와 Maxpooling 으로 구성된 특징 추출기만 가져온 것을 알 수 있습니다.

In [23]:
md = Sequential()
md.add(base_md)
#pre_trained model
md.add(Flatten())
#Flatten - 텐서를 1차원 벡터로 변환시켜줌, 사전 학습된 특징 추출기와 분류기를 연결시켜주는 역할
md.add(Dense(64, activation = 'relu'))
md.add(Dropout(0.25))
md.add(Dense(4, activation = 'softmax'))

md.compile(loss = 'sparse_categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(),\
          metrics = ['accuracy'])
md.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_5 (Flatten)         (None, 25088)             0         
                                                                 
 dense_17 (Dense)            (None, 64)                1605696   
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_18 (Dense)            (None, 4)                 260       
                                                                 
Total params: 16,320,644
Trainable params: 16,320,644
Non-trainable params: 0
_________________________________________________________________


----------------------------------------------------------------------------------------------

- 전이 학습 예제(Cats and Dogs) -

Cats and Dogs Dataset : CNN 아키텍쳐 구축 및 평가를 위한 기본 학습 데이터셋

2000 개의 학습 데이터, 1000 개의 테스트 데이터로 구성, 학습 데이터가 부족하기 때문에 전이 학습을 활용!

In [25]:
#Fine-tuning using pre-trained model(Xception)

from tensorflow.keras.applications import Xception
#사전 학습 모델 import
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Cats and Dogs 데이터셋 압축풀기
import zipfile

with zipfile.ZipFile('cats_and_dogs_filtered.zip', 'r') as target_file:

    target_file.extractall()

In [28]:
#모델링(pre-trained Xception + User-Defined Classifier)

img_width = 224
img_height = 224

base_md = Xception(weights = 'imagenet', include_top = False, input_shape = (img_width, img_height, 3))
#사전 학습 모델의 feature extractor 만 사용(include_top = False)

md = Sequential()

md.add(base_md)

md.add(GlobalAveragePooling2D())
#------------------------------------------- User-Defined Classifier(아래)
md.add(Dense(16, activation = 'relu'))
md.add(Dropout(0.25))

md.add(Dense(2, activation = 'softmax'))
#정답은 고양이 또는 개, 총 두개이므로 출력층 노드 : 2
md.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_19 (Dense)            (None, 16)                32784     
                                                                 
 dropout_10 (Dropout)        (None, 16)                0         
                                                                 
 dense_20 (Dense)            (None, 2)                 34        
                                                                 
Total params: 20,894,298
Trainable params: 20,839,770
Non-trainable params: 54,528
_____________________________________

In [29]:
#ImageDataGenerator 정의

train_dir = './cats_and_dogs_filtered/train'
test_dir = './cats_and_dogs_filtered/validation'

train_data_gen = ImageDataGenerator(rescale = 1./ 255, 
                                    rotation_range = 10, width_shift_range = 0.1, 
                                    height_shift_range = 0.1, shear_range = 0.1, zoom_range = 0.1)

test_data_gen = ImageDataGenerator(rescale = 1./255)
#이미지 데이터 읽어올 때, 자동으로 정규화

In [30]:
train_data = train_data_gen.flow_from_directory(train_dir, batch_size = 32, 
                                                color_mode = 'rgb', shuffle = True, class_mode = 'categorical',
                                                target_size = (img_width,img_height))


test_data = test_data_gen.flow_from_directory(test_dir, batch_size = 32, 
                                              color_mode = 'rgb', shuffle = True, class_mode = 'categorical',
                                              target_size = (img_width,img_height))

#class_mode = 'categorical' : 정답은 원핫 인코딩으로 정의됨.

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [31]:
#정답 확인

print(train_data.class_indices.items())
print(test_data.class_indices.items())
#class_indices : 문자열로 표시되는 데이터 정답이 어떤 숫자로 매칭되는지 확인할 수 있음.

dict_items([('cats', 0), ('dogs', 1)])
dict_items([('cats', 0), ('dogs', 1)])


In [ ]:
#모델 컴파일 및 학습

from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

md.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(2e-5),\
          metrics = ['accuracy'])
#fine-tuning : 학습률을 낮게 설정해 pre-trained 가중치를 조금씩 업데이트 해주는 것이 핵심,

save_file_name = './cats_and_dogs_Xception'

checkpoint = ModelCheckpoint(save_file_name, monitor = 'val_loss',\
                            verbose = 1, save_best_only = True, mode = 'auto')

es = EarlyStopping(monitor = 'val_loss', patience = 5)

hist = md.fit(train_data, epochs = 30, validation_data = test_data, callbacks = [checkpoint, es])

---------------------------------------------------------------------------------------------------

Callback Function)

특정 상황에서 실행되는 함수를 시스템에 등록 -> 해당 상황이 발생됐을 때, 등록된 함수가 실행

ReduceLROnPlateau - 학습 중에 학습률 변화시킬 수 있는 함수

In [ ]:
#example)

from tensorflow.keras.callbacks import ReduceLROnPlateau

reduceLR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5,
                            #val_loss 기준으로 callback 출력 #callback 호출시, 학습률을 1/2 줄임
                            patience = 5, verbose = 1)
                            #epoch 5 동안 개선되지 않으면 callback 호출, 로그 출력

hist = md.fit(xtr, ytr, epochs = 50, validation_split = 0.2, callbacks = [reduceLR])

ModelCheckpoint - 모델 가중치를 중간에 저장할 수 있는 함수

In [ ]:
#example)

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = Modelcheckpoint(file_path, monitor = 'val_loss', verbose = 1,
                            #val_loss 값이 개선됐을 때 호출, 로그 출력
                            save_best_only = True, mode = 'auto')
                            #best 값만 저장, 자동으로 best 값을 찾음.
                          

hist = md.fit(xtr, ytr, epochs = 50, validation_split = 0.2, callbacks = [checkpoint])

EarlyStopping - 모델 성능지표가 일정 시간 개선되지 않을 때 조기 종료시킬 수 있는 함수

In [ ]:
#example)

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = Modelcheckpoint(file_path, monitor = 'val_loss', verbose = 1,
                            #val_loss 값이 개선됐을 때 호출, 로그 출력
                            save_best_only = True, mode = 'auto')
                            #best 값만 저장, 자동으로 best 값을 찾음.
    
es = EarlyStopping(monitor = 'val_loss', patience = 5)
                  #관찰 대상은 val_loss, epochs = 5 동안 개선되지 않으면 학습 종료
                          

hist = md.fit(xtr, ytr, epochs = 50, validation_split = 0.2, callbacks = [checkpoint, es])